In [1]:
%load_ext autoreload

In [131]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import seaborn as sns
import numpy as np

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.grid_search import GridSearchCV

import transform_utils as tu
from kaggle_helper import KaggleHelper

In [120]:
kh = KaggleHelper("submissions.db", "predictions")

table submissions already exists
table validation_scores already exists


In [130]:
%aimport transform_utils
%aimport kaggle_helper

In [116]:
%aimport

Modules to reload:
kaggle_helper transform_utils

Modules to skip:



In [117]:
raw = pd.read_csv("data/train.csv")
print("Unique UPC:", len(raw.Upc.unique()))
print("Unique DepartmentDescription:", len(raw.DepartmentDescription.unique()))
print("Unique FinelineNumber:", len(raw.FinelineNumber.unique()))
raw.head()

Unique UPC: 97715
Unique DepartmentDescription: 69
Unique FinelineNumber: 5196


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [129]:
start = datetime.datetime.now()
%autoreload
df = tu.preprocess_walmart(raw, True)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

Total Time: 4.125009


In [121]:
X = df.drop("TripType", axis=1).drop("VisitNumber", axis=1)
sorted_columns = sorted(X.columns)
y = df.TripType

In [125]:
XTrain, XTest, yTrain, yTest = train_test_split(X.values, y.values, test_size=0.2, random_state=200)
XTrain, XVal, yTrain, yVal = train_test_split(XTrain, yTrain, test_size=0.2, random_state=200)

http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV

In [126]:
# for x in [50, 75, 100]:
#     for train, test in KFold(len(a_train), 3):
#         clf = RandomForestClassifier(n_estimators=250, n_jobs=3, max_depth=x)
#         clf.fit(a_train[train], b_train[train])
#         print("max depth:", x, clf.score(a_train[test], b_train[test]))

In [132]:
start = datetime.datetime.now()

clf = RandomForestClassifier(n_jobs=3)
clf.fit(XTrain, yTrain)
score = clf.score(XVal, yVal)

print(score)
print("Total Time:", (datetime.datetime.now() - start).total_seconds())

0.652273321139
Total Time: 1.655616


In [133]:
kh.save_validation_score(clf, score, len(XTrain))

Enter Notes to save to DB: returns and scancount feature engineering


# Only Kaggle Exists Below Here

In [134]:
kaggleTest = tu.preprocess_walmart(pd.read_csv("data/test.csv"))
missing_columns = set(sorted_columns).difference(set(kaggleTest.columns))
print("missing columns")
print("-"*15)
for col in missing_columns:
    print(col)
    kaggleTest[col] = 0
kaggleTest = kaggleTest[sorted_columns]

missing columns
---------------
HEALTH AND BEAUTY AIDS


In [135]:
test_predictions = clf.predict(kaggleTest.values)
output = tu.convert_predictions(test_predictions, 
                               output_index=pd.Series(kaggleTest.index), 
                               actual_trip_types=df.TripType.unique())

Missing Trip Types: ['TripType_32', 'TripType_33', 'TripType_34', 'TripType_35', 'TripType_37', 'TripType_41', 'TripType_43', 'TripType_44', 'TripType_14', 'TripType_15', 'TripType_20', 'TripType_23', 'TripType_26', 'TripType_27']


In [136]:
kh.save_test_predictions(output, clf, len(XTrain))

Enter Notes to save to DB:  Missing Trip Types: ['TripType_32', 'TripType_33', 'TripType_34', 'TripType_35', 'TripType_37', 'TripType_41', 'TripType_43', 'TripType_44', 'TripType_14', 'TripType_15', 'TripType_20', 'TripType_23', 'TripType_26', 'TripType_27'], with scan count feature
